Dentro de este Notebook se realizará la lógica necesaria para la generación de datos aleatorios necesarios para posteriores desarrollos

In [0]:
# librerias utilizadas

import copy
import numpy as np
import pandas as pd
from google.colab import drive
from random import uniform

In [2]:
#nos conectamos a las plantillas generadas en csv

drive.mount('/gdrive')
analitica_template = pd.read_excel('/gdrive/My Drive/Pathologies Study/COVID-Study/templates/analitica_plantilla.xlsx')
analitica_range = pd.read_excel('/gdrive/My Drive/Pathologies Study/COVID-Study/templates/analitica_rangos.xlsx')
anamnesis_template = pd.read_excel('/gdrive/My Drive/Pathologies Study/COVID-Study/templates/anamnesis_plantilla.xlsx')
anamnesis_range = pd.read_excel('/gdrive/My Drive/Pathologies Study/COVID-Study/templates/anamnesis_rangos.xlsx')

KeyboardInterrupt: ignored

In [0]:
#definimos los rangos de sobreinfección y transtorno de la coagulación

sobreinfec = pd.DataFrame([["procalcitonina", 1, 2]],columns= ["Parámetro", "minimo", "maximo"])
trans_coa = pd.DataFrame([["d_dímero", 1000, 1500], ["nt-probnp", 125, 200]],columns= ["Parámetro", "minimo", "maximo"])

In [0]:
# definimos los diferentes escenarios para generar los valores aleatorios

clusters = ["Inflamación Intensa", "Inflamación Moderada", "Inflamación Leve"]
patron_adiccional = ["Ninguno", "SobreInfección", "Transtorno Coagulación"]

In [0]:
# definimos una función para generar valores aleatorios (establecemos un 20% de probabilidades de estar dentro de sobreinfección y un 20% de probabilidades de tener transtorno de coagulación)

def generate_random_values(cluster):
    template = copy.deepcopy(analitica_range)

    #definimos los aleatorios a realizar en función del cluster introducido
    if cluster == "Inflamación Intensa":
        template['Value'] = template.apply(lambda x: round(uniform(x.Mínimo_1, x.Máximo_1), 2), axis = 1)
    elif cluster == "Inflamación Moderada":
        template['Value'] = template.apply(lambda x: round(uniform(x.Mínimo_2, x.Máximo_2), 2), axis = 1)
    elif cluster == "Inflamación Leve":
        template['Value'] = template.apply(lambda x: round(uniform(x.Mínimo_3, x.Máximo_3), 2), axis = 1)
    else:
        raise Exception('El cluster introducido no es válido, debe de estar dentro del rango: Inflamación Intensa, Inflamación Moderada, Inflamación Leve')


    #eliminamos las columnas que ya no son necesarias
    template.drop(['Mínimo_1', 'Máximo_1', 'Mínimo_2', 'Máximo_2', 'Mínimo_3', 'Máximo_3'], axis = 1, inplace = True)

    #establecemos un 20% de probabilidades de tener sobreinfección
    proba_sobr = uniform(0, 1)
    if proba_sobr >= 0.8:
        template['Value'] = template.apply(lambda x: round(uniform(sobreinfec['minimo'][0], sobreinfec['maximo'][0]), 2) if x.Parámetro == sobreinfec['Parámetro'][0] else x.Value, axis = 1)

    #establecemos un 20% de probabilidades de tener transtorno de coagulación
    proba_coag = uniform(0, 1)
    if proba_coag >= 0.8:
        for i in range(trans_coa.shape[0]):
          template['Value'] = template.apply(lambda x: round(uniform(trans_coa['minimo'][i], trans_coa['maximo'][i]), 2) if x.Parámetro == trans_coa['Parámetro'][i] else x.Value, axis = 1)

    #modificamos los datos obtenidos para poder introducirlos en el sistema
    template = template.set_index(['Parámetro']).transpose().reset_index().drop(['index'], axis = 1)
    template.index.names = [""]
    template['Cluster'] = cluster
    return template

In [0]:
#ejecutamos por primera vez el generador de datos para obtener el DataFrame base

data = generate_random_values(clusters[0])

In [0]:
#generamos 200 muestras de datos para cada uno de los cluster

for j in range(len(clusters)):
  for i in range(200):
      data = data.append(generate_random_values(clusters[j]))

In [0]:
data

**En los siguientes DataFrames podemos visualizar diferentes muestras de los datos aleatorios generados**

Muestra de pacientes con Inflamación Intensa

In [0]:
data[data['Cluster'] == "Inflamación Intensa"].head()

Muestra de pacientes con Inflamación Moderada

In [0]:
data[data['Cluster'] == "Inflamación Moderada"].head()

Muestra de pacientes con Inflamación Leve

In [0]:
data[data['Cluster'] == "Inflamación Leve"].head()

In [0]:
#exportamos los datos que utilizaremos en otros scripts

data.to_csv('/gdrive/My Drive/Pathologies Study/COVID-Study/data/random_data.csv', sep = ";", encoding = 'latin-1', index = False)